In [430]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

In [431]:
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

In [432]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

In [433]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

In [434]:
US_ED = pd.read_sql_query('select * from useducation',con=engine)

In [435]:
engine.dispose()

In [454]:
US_ED.iloc[:, 4:29].head()

,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [455]:
US_ED1= US_ED.iloc[:, 4:28]

In [456]:
US_ED1= US_ED1.interpolate()

In [457]:
US_ED1=US_ED1.dropna()

In [458]:
US_ED1.isnull().count()

TOTAL_REVENUE                   1456
FEDERAL_REVENUE                 1456
STATE_REVENUE                   1456
LOCAL_REVENUE                   1456
TOTAL_EXPENDITURE               1456
INSTRUCTION_EXPENDITURE         1456
SUPPORT_SERVICES_EXPENDITURE    1456
OTHER_EXPENDITURE               1456
CAPITAL_OUTLAY_EXPENDITURE      1456
GRADES_PK_G                     1456
GRADES_KG_G                     1456
GRADES_4_G                      1456
GRADES_8_G                      1456
GRADES_12_G                     1456
GRADES_1_8_G                    1456
GRADES_9_12_G                   1456
GRADES_ALL_G                    1456
AVG_MATH_4_SCORE                1456
AVG_MATH_8_SCORE                1456
AVG_READING_4_SCORE             1456
AVG_READING_8_SCORE             1456
dtype: int64

# 1.Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [381]:
'''#def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()'''

'#def wavg(group, avg_name, weight_name):\n    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns\n    In rare instance, we may not have weights, so just return the mean. Customize this if your business case\n    should return otherwise.\n    """\n    d = group[avg_name]\n    w = group[weight_name]\n    try:\n        return (d * w).sum() / w.sum()\n    except ZeroDivisionError:\n        return d.mean()'

In [382]:
#US_ED1["weight"] = (US_ED1["AVG_MATH_4_SCORE"] +
#              US_ED1["AVG_MATH_8_SCORE"] + US_ED1["AVG_READING_4_SCORE"] + US_ED1["AVG_READING_8_SCORE"])/4

In [443]:
US_ED1["overall_score"] = (US_ED1["GRADES_4_G"]*((US_ED1["AVG_MATH_4_SCORE"] + US_ED1["AVG_READING_4_SCORE"])*0.5) + US_ED1["GRADES_8_G"]
                                 * ((US_ED1["AVG_MATH_8_SCORE"] + US_ED1["AVG_READING_8_SCORE"])*0.5))/(US_ED1["GRADES_4_G"] + US_ED1["GRADES_8_G"])


In [446]:
US_ED1.head()

,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,...,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,overall_score
36,804297.0,69172.0,403021.0,332104.0,787949.0,426072.0,254937.0,24913.0000,67913.0,7379.5,...,7211.0,6855.0,53091.0,29035.0,1098988.5,219.515870,268.128094,221.250787,261.870243,243.700100
37,2396705.0,211627.0,1470516.0,714562.0,2515272.0,1283577.0,779828.0,38182.3125,194081.0,4859.0,...,44163.0,33346.0,387659.0,157824.0,550342.0,220.319806,268.131990,219.256638,261.861030,241.266487
38,2773959.0,163544.0,788309.0,1822106.0,2898210.0,1556770.0,967075.0,51451.6250,230464.0,771.0,...,39610.0,31920.0,325128.0,144002.0,469901.0,222.312769,269.755214,217.262490,261.851817,242.275252
39,11257252.0,658139.0,4227323.0,6371790.0,11539253.0,6075381.0,3329224.0,64720.9375,964457.0,4416.0,...,126293.0,108244.0,1063552.0,486354.0,1554322.0,224.305732,271.378439,215.268341,261.842604,242.748345
40,883073.0,43545.0,329810.0,509718.0,863404.0,556787.0,265279.0,77990.2500,14685.0,498.0,...,10204.0,8244.0,90758.0,37931.0,129187.0,215.449248,265.907109,219.940098,261.833392,239.781374


# 2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [447]:
US_ED1= US_ED1.iloc[:, np.r_[4:9, -1]]

In [448]:
US_ED1.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,overall_score
TOTAL_EXPENDITURE,1.000000,0.992698,0.992435,0.951726,0.928129,0.204239
INSTRUCTION_EXPENDITURE,0.992698,1.000000,0.979165,0.920297,0.895527,0.207131
SUPPORT_SERVICES_EXPENDITURE,0.992435,0.979165,1.000000,0.953411,0.905265,0.221824
OTHER_EXPENDITURE,0.951726,0.920297,0.953411,1.000000,0.923468,0.169775
CAPITAL_OUTLAY_EXPENDITURE,0.928129,0.895527,0.905265,0.923468,1.000000,0.128087
overall_score,0.204239,0.207131,0.221824,0.169775,0.128087,1.000000


In [423]:
#support services expenditure has the highest correlation with overall score. All of the expenditures are 
#highly correlated with each other. 

# 3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [450]:
#practicing standardization manually
'''US_ED1['CAPITAL_OUTLAY_EXPENDITURE_Z'] = (US_ED1['CAPITAL_OUTLAY_EXPENDITURE'] - US_ED1['CAPITAL_OUTLAY_EXPENDITURE'].mean()) / US_ED1['CAPITAL_OUTLAY_EXPENDITURE'].std()
US_ED1['INSTRUCTION_EXPENDITURE_Z'] = (US_ED1['INSTRUCTION_EXPENDITURE'] - US_ED1['INSTRUCTION_EXPENDITURE'].mean()) / US_ED1['INSTRUCTION_EXPENDITURE'].std()
US_ED1['SUPPORT_SERVICES_EXPENDITURE_Z'] = (US_ED1['SUPPORT_SERVICES_EXPENDITURE'] - US_ED1['SUPPORT_SERVICES_EXPENDITURE'].mean()) / US_ED1['SUPPORT_SERVICES_EXPENDITURE'].std()
US_ED1['OTHER_EXPENDITURE_Z'] = (US_ED1['OTHER_EXPENDITURE'] - US_ED1['OTHER_EXPENDITURE'].mean()) / US_ED1['OTHER_EXPENDITURE'].std()


X=["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]'''

'US_ED1[\'CAPITAL_OUTLAY_EXPENDITURE_Z\'] = (US_ED1[\'CAPITAL_OUTLAY_EXPENDITURE\'] - US_ED1[\'CAPITAL_OUTLAY_EXPENDITURE\'].mean()) / US_ED1[\'CAPITAL_OUTLAY_EXPENDITURE\'].std()\nUS_ED1[\'INSTRUCTION_EXPENDITURE_Z\'] = (US_ED1[\'INSTRUCTION_EXPENDITURE\'] - US_ED1[\'INSTRUCTION_EXPENDITURE\'].mean()) / US_ED1[\'INSTRUCTION_EXPENDITURE\'].std()\nUS_ED1[\'SUPPORT_SERVICES_EXPENDITURE_Z\'] = (US_ED1[\'SUPPORT_SERVICES_EXPENDITURE\'] - US_ED1[\'SUPPORT_SERVICES_EXPENDITURE\'].mean()) / US_ED1[\'SUPPORT_SERVICES_EXPENDITURE\'].std()\nUS_ED1[\'OTHER_EXPENDITURE_Z\'] = (US_ED1[\'OTHER_EXPENDITURE\'] - US_ED1[\'OTHER_EXPENDITURE\'].mean()) / US_ED1[\'OTHER_EXPENDITURE\'].std()\n\n\nX=["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",\n                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]'

In [451]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = US_ED1[["INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
US_ED1["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)


The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.94725496]


# 4.What is the correlation between the overall score variable and the 1st principal component?

In [453]:
US_ED1.corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,overall_score,pca_1
TOTAL_EXPENDITURE,1.000000,0.992698,0.992435,0.951726,0.928129,0.204239,0.992988
INSTRUCTION_EXPENDITURE,0.992698,1.000000,0.979165,0.920297,0.895527,0.207131,0.975096
SUPPORT_SERVICES_EXPENDITURE,0.992435,0.979165,1.000000,0.953411,0.905265,0.221824,0.986139
OTHER_EXPENDITURE,0.951726,0.920297,0.953411,1.000000,0.923468,0.169775,0.975451
CAPITAL_OUTLAY_EXPENDITURE,0.928129,0.895527,0.905265,0.923468,1.000000,0.128087,0.956156
overall_score,0.204239,0.207131,0.221824,0.169775,0.128087,1.000000,0.187067
pca_1,0.992988,0.975096,0.986139,0.975451,0.956156,0.187067,1.000000


In [ ]:
#it's a low correlation at 0.187

# 5.If you were to choose the best variables for your model, would you prefer using the first principal component instead of the expenditure variables? Why?


In [ ]:
#support services expenditure, instruction and other expenditures have the higer correlation than PCA.We can use 
#expenditure variables instead